برای اجرای نوت‌بوک‌های زیر، اگر تا کنون این کار را نکرده‌اید، باید کلید openai را به عنوان `OPENAI_API_KEY` در فایل .env قرار دهید.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

در مرحله بعد، قصد داریم ایندکس تعبیه را در یک دیتافریم پانداس بارگذاری کنیم. ایندکس تعبیه در یک فایل JSON به نام `embedding_index_3m.json` ذخیره شده است. ایندکس تعبیه شامل تعبیه‌ها برای هر یک از رونوشت‌های یوتیوب تا اواخر اکتبر ۲۰۲۳ است.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

در ادامه، قصد داریم تابعی به نام `get_videos` بسازیم که در ایندکس تعبیه (Embedding Index) به دنبال پرس‌وجو (query) جستجو می‌کند. این تابع، ۵ ویدیویی که بیشترین شباهت را به پرس‌وجو دارند، برمی‌گرداند. عملکرد این تابع به این صورت است:

1. ابتدا یک کپی از ایندکس تعبیه ساخته می‌شود.
2. سپس تعبیه (Embedding) مربوط به پرس‌وجو با استفاده از OpenAI Embedding API محاسبه می‌شود.
3. بعد، یک ستون جدید به نام `similarity` در ایندکس تعبیه ایجاد می‌شود. ستون `similarity` شامل میزان شباهت کسینوسی بین تعبیه پرس‌وجو و تعبیه هر بخش از ویدیو است.
4. سپس ایندکس تعبیه بر اساس ستون `similarity` فیلتر می‌شود. فقط ویدیوهایی که شباهت کسینوسی آن‌ها برابر یا بیشتر از ۰.۷۵ باشد، نگه داشته می‌شوند.
5. در نهایت، ایندکس تعبیه بر اساس ستون `similarity` مرتب شده و ۵ ویدیوی برتر بازگردانده می‌شود.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

این تابع بسیار ساده است، فقط نتایج جستجو را چاپ می‌کند.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

۱. ابتدا ایندکس تعبیه‌سازی در یک دیتافریم پانداس بارگذاری می‌شود.
۲. سپس از کاربر خواسته می‌شود یک پرس‌وجو وارد کند.
۳. بعد تابع `get_videos` فراخوانی می‌شود تا ایندکس تعبیه‌سازی را برای پرس‌وجو جستجو کند.
۴. در نهایت، تابع `display_results` فراخوانی می‌شود تا نتایج را به کاربر نمایش دهد.
۵. سپس دوباره از کاربر خواسته می‌شود یک پرس‌وجوی دیگر وارد کند. این روند ادامه پیدا می‌کند تا زمانی که کاربر عبارت `exit` را وارد کند.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.fa.png)

از شما خواسته می‌شود یک پرس‌وجو وارد کنید. یک پرس‌وجو وارد کنید و اینتر بزنید. برنامه فهرستی از ویدیوهای مرتبط با پرس‌وجوی شما را نمایش می‌دهد. همچنین لینکی به بخشی از ویدیو که پاسخ سؤال شما در آن قرار دارد ارائه می‌شود.

در اینجا چند نمونه پرس‌وجو برای امتحان کردن آورده شده است:

- Azure Machine Learning چیست؟
- شبکه‌های عصبی کانولوشنی چگونه کار می‌کنند؟
- شبکه عصبی چیست؟
- آیا می‌توانم از Jupyter Notebooks با Azure Machine Learning استفاده کنم؟
- ONNX چیست؟


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطا یا نادقتی باشند. نسخه اصلی سند به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نداریم.
